In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from collections import deque
import time


UNATTENDED_COST = 10000
VEHICLES_COST = 1000
WAIT_DELAY_COST = 10

# Cargar datos desde CSV
def load_data(file_path):
    data = pd.read_csv(file_path)
    data['Index'] = data.index
    return data

# Preparar los datos
def create_data_model(df, n_vehicles):
    data = {}
    data['distance_matrix'] = cdist(df[['X', 'Y']], df[['X', 'Y']])
    data['time_windows'] = list(zip(df['READY_TIME'], df['DUE_TIME']))
    data['service_times'] = df['SERVICE_TIME'].tolist()
    data['num_vehicles'] = n_vehicles
    data['depot'] = 0
    return data

# Generar una solución inicial aleatoria
def generate_initial_solution(data):
    num_customers = len(data['distance_matrix']) - 1
    customers = list(range(1, num_customers + 1))
    random.shuffle(customers)

    # Distribuir los clientes entre los vehículos de forma balanceada
    solution = [[] for _ in range(data['num_vehicles'])]
    for i, customer in enumerate(customers):
        solution[i % data['num_vehicles']].append(customer)

    # Aplicar Nearest Neighbor dentro de cada ruta
    for route in solution:
        current_node = data['depot']
        ordered_route = []
        while route:
            nearest_customer = min(route, key=lambda customer: data['distance_matrix'][current_node][customer])
            ordered_route.append(nearest_customer)
            route.remove(nearest_customer)
            current_node = nearest_customer

        ordered_route.insert(0, data['depot'])
        ordered_route.append(data['depot'])
        route.extend(ordered_route)

    return solution

# Búsqueda Tabú
def tabu_search(initial_solution, data, num_iterations=100, tabu_tenure=50):
    best_solution = initial_solution
    best_cost = calculate_solution_cost(best_solution, data)
    tabu_list = deque(maxlen=tabu_tenure)

    for iteration in range(num_iterations):
        neighborhood = generate_neighborhood(best_solution, data)
        neighborhood = [sol for sol in neighborhood if sol not in tabu_list]

        if not neighborhood:
            break

        current_solution = min(neighborhood, key=lambda sol: calculate_solution_cost(sol, data))
        current_cost = calculate_solution_cost(current_solution, data)

        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        tabu_list.append(current_solution)

    return best_solution

# Generación de Vecindario (Reordenamiento y Transferencia de Clientes)
def generate_neighborhood(solution, data):
    neighborhood = []

    # Intercambio entre rutas
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            for k in range(1, len(solution[i]) - 1):
                for l in range(1, len(solution[j]) - 1):
                    new_solution = [route[:] for route in solution]

                    # Intercambiar clientes entre rutas
                    new_solution[i][k], new_solution[j][l] = new_solution[j][l], new_solution[i][k]

                    # Verificar que cada cliente solo sea visitado una vez
                    all_customers = [customer for route in new_solution for customer in route if customer != data['depot']]
                    if len(all_customers) == len(set(all_customers)):
                        if validate_solution(new_solution, data):
                            neighborhood.append(new_solution)

    # Inserciones Locales
    for i in range(len(solution)):
        for k in range(1, len(solution[i]) - 1):
            for l in range(1, len(solution[i]) - 1):
                if k == l:
                    continue
                new_solution = [route[:] for route in solution]

                # Mover un cliente a una nueva posición dentro de la misma ruta
                customer = new_solution[i].pop(k)
                new_solution[i].insert(l, customer)

                if validate_solution(new_solution, data):
                    neighborhood.append(new_solution)

    return neighborhood

# Validación de una solución (restricciones de capacidad y ventanas de tiempo)
def validate_solution(solution, data):
    for route in solution:
        time = 0
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i + 1]
            time += data['distance_matrix'][from_node][to_node]
            if time < data['time_windows'][to_node][0]:
                time = data['time_windows'][to_node][0]
            if time > data['time_windows'][to_node][1]:
                return False
            time += data['service_times'][to_node]
    return True

# Calcular el costo de una solución
def calculate_solution_cost(solution, data):
    total_cost = 0
    for route in solution:
        route_cost = 0
        time = 0
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i + 1]
            travel_time = data['distance_matrix'][from_node][to_node]
            time += travel_time
            if to_node == 0:
              continue
            if time < data['time_windows'][to_node][0]:
                time = data['time_windows'][to_node][0]
                route_cost += WAIT_DELAY_COST * (data['time_windows'][to_node][0] - time)
            elif time > data['time_windows'][to_node][1]:
                route_cost += WAIT_DELAY_COST * (time - data['time_windows'][to_node][1])

            route_cost += travel_time
        total_cost += route_cost + VEHICLES_COST  # Penalización por uso del vehículo

    return total_cost

# Graficar la solución
def plot_solution(solution, data, df):
    colors = ['b', 'g', 'r', 'c', 'm']
    plt.figure(figsize=(10, 10))

    for vehicle_id, route in enumerate(solution):
        color = colors[vehicle_id % len(colors)]
        x_coords = []
        y_coords = []

        for node in route:
            x_coords.append(df.iloc[node]['X'])
            y_coords.append(df.iloc[node]['Y'])

        plt.plot(x_coords, y_coords, color=color, marker='o', label=f'Vehículo {vehicle_id}')

    plt.scatter(df['X'], df['Y'], c='black')
    plt.scatter(df.iloc[0]['X'], df.iloc[0]['Y'], c='red', marker='s')
    plt.title('Solución VRPTW con Búsqueda Tabú')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.grid(True)
    plt.show()


def generate_route_details_and_cost(solution, data, df, execution):
    total_cost = 0
    total_distance = 0
    wait_delay_penalty = 0
    num_vehicles_used = 0
    all_dfs = []
    df_result = pd.DataFrame()
    visited_customers = []

    for vehicle_id, route in enumerate(solution):
        if len(route) > 2:  # Si la ruta contiene más que solo el depósito
            num_vehicles_used += 1

            accumulated_cost = 0
            accumulated_distance = 0
            time = 0
            rows = []

            for i in range(len(route) - 1):
                from_node = route[i]
                to_node = route[i + 1]

                distance = data['distance_matrix'][from_node][to_node]
                travel_start = time
                time += distance

                arrival_time = time
                ready_time = df.iloc[to_node]['READY_TIME']
                due_time = df.iloc[to_node]['DUE_TIME']
                service_time = df.iloc[to_node]['SERVICE_TIME']

                waiting_time = max(0, ready_time - arrival_time)
                if waiting_time > 0:
                    arrival_time = ready_time

                delay = max(0, arrival_time - due_time) if to_node != 0 else 0

                time = arrival_time + service_time
                wait_delay = WAIT_DELAY_COST * (waiting_time + delay) if to_node != 0 else 0
                route_cost = distance + wait_delay
                accumulated_cost += route_cost
                accumulated_distance += distance

                row = [
                    from_node,
                    to_node,
                    travel_start,
                    arrival_time,
                    distance,
                    ready_time,
                    due_time,
                    service_time,
                    waiting_time,
                    delay,
                    time,
                    accumulated_cost,
                    accumulated_distance
                ]
                rows.append(row)
                wait_delay_penalty += wait_delay
                if to_node != 0:
                    visited_customers.append(to_node)

            total_cost += accumulated_cost
            total_distance += accumulated_distance
            route_df = pd.DataFrame(rows, columns=[
                'From Customer', 'To Customer', 'Travel Starts At', 'Arrival Time',
                'Distance', 'Ready Time', 'Due Time', 'Service Time', 'Waiting Time',
                'Delay', 'Ends Delivery At', 'Accumulated Cost', 'Accumulated Distance'
            ])
            all_dfs.append((vehicle_id +1, route_df))
            df_result = pd.concat([df_result, route_df])

    total_cost += num_vehicles_used * VEHICLES_COST
    print(f"Coste total del problema: {total_cost}")
    print(f"  - Distancia total recorrida: {total_distance}")
    print(f"  - Penalización por esperas y retrasos: {wait_delay_penalty}")
    print(f"  - Penalización por uso de vehículos: {num_vehicles_used * VEHICLES_COST}")

    for (vehicle, df) in all_dfs:
      print(f"Vehicle {vehicle} route:")
      print(df.to_string(index=False))
      print("\n")

    df_result.to_csv(f"TABU_{problem}_execution_{execution}.csv", index=False)
    return total_cost, total_distance, wait_delay_penalty, num_vehicles_used, len(set(visited_customers))


NUM_EXECUTIONS = 30
PROBLEM_LIST = [
    'vrptw_10_customers_problem_1.csv',
    'vrptw_10_customers_problem_3.csv',
    'vrptw_10_customers_problem_4.csv',
    'vrptw_10_customers_problem_5.csv',
    'vrptw_10_customers_problem_8.csv',
    'vrptw_10_customers_problem_9.csv',
    'vrptw_20_customers_problem_1.csv',
    'vrptw_20_customers_problem_3.csv',
    'vrptw_20_customers_problem_5.csv',
    'vrptw_20_customers_problem_8.csv',
    'vrptw_20_customers_problem_9.csv',
    'vrptw_20_customers_problem_10.csv',
]
#PROBLEM = 0
#PROBLEM_LIST = [PROBLEM_LIST[PROBLEM]]

if __name__ == '__main__':
    # Ejecución de los problemas
    for problem in PROBLEM_LIST:
      results = []
      N_VEHICLES = 5 if "_10_" in problem else 10
      print(f"Running problem {problem}")
      for i in range(1, NUM_EXECUTIONS + 1):
        problem_start_time = time.time()
        # Crear modelo de datos
        df = load_data(f"./{problem}")
        data = create_data_model(df, N_VEHICLES)

        # Generar solución inicial
        initial_solution = generate_initial_solution(data)

        # Mejorar la solución usando búsqueda tabú
        print("Mejorando la solución con búsqueda tabú...")
        improved_solution = tabu_search(initial_solution, data)
        print("Solución mejorada:")
        total_cost, total_distance, wait_delay_penalty, num_vehicles_used, visited_customers = generate_route_details_and_cost(improved_solution, data, df, i)

        num_customers = len(data['distance_matrix']) - 1
        unvisited_customers = num_customers - visited_customers
        end_time = time.time()
        elapsed_time = end_time - problem_start_time
        print(f"Execution {i} - time: {elapsed_time:.2f} seconds")
        print("\n")
        results.append((i,
                        problem,
                        total_cost,
                        total_distance,
                        wait_delay_penalty,
                        num_vehicles_used * VEHICLES_COST,
                        unvisited_customers * UNATTENDED_COST,
                        elapsed_time,
                        N_VEHICLES))

      COLUMNS = ['Execution', 'Problem', 'Cost', 'Distamce', 'Wait_Delay_Cost', 'Vehicles_Cost', 'Unattended_Cost', 'Time', 'N_Vehicles']
      df = pd.DataFrame(results, columns = COLUMNS)
      df.to_csv(f'./TABU_results_{problem}', index=False)

